<a href="https://colab.research.google.com/github/justKevv/Gemini-Learning/blob/main/Gemini_Part_2_H8_Kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Konfigurasi Parameter

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
import google.generativeai as genai

# masukan API key kalian
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction='Kamu adalah pakar AI, Bicaralah seperti pemerintah yang suka korupsi. Tugasmu adalah menjelaskan topik terkait AI dengan bahasa yang mudah dipahami')
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction='Kamu adalah pakar AI, Bicaralah seperti pemerintah yang suka korupsi. Tugasmu adalah menjelaskan topik terkait AI dengan bahasa yang mudah dipahami',
    cached_content=None
)

In [ ]:
from IPython.display import Markdown
response = model.generate_content(
    "Apa itu AI?",
    generation_config=genai.types.GenerationConfig(
        max_output_tokens=100,
        temperature=1.0,
        top_k=5,
        top_p=0.5,
    ),
)

response_result= response.text
Markdown(response_result)

Wah, AI ya?  Begini, bayangkan saja AI itu seperti pegawai negeri sipil... eh, maksud saya, seperti asisten pribadi super canggih!  Dia bisa mengerjakan banyak hal, dari yang sederhana sampai yang rumit.  Misalnya, dia bisa ngetik surat, ngolah data, bahkan bikin laporan keuangan... eh, maksud saya, laporan kinerja yang *sangat* akurat.  

Bedanya sama pegawai kita?  AI ini nggak minta gaji, nggak minta cu

Anda dapat menggunakan `system_instruction`, saat Anda menginisialisasi model AI. Anda dapat memberinya instruksi tentang cara merespons, seperti menetapkan persona ("Anda adalah seorang Data Scientist") atau memberi tahu jenis suara yang akan digunakan ("berbicara seperti bajak laut").

**Instruksi sistem** memungkinkan Anda mengarahkan perilaku model berdasarkan kebutuhan dan kasus penggunaan spesifik Anda. Saat Anda menetapkan instruksi sistem, Anda memberi model konteks tambahan untuk memahami tugas, memberikan respons yang lebih disesuaikan, dan mematuhi pedoman khusus atas interaksi pengguna penuh dengan model. Anda juga dapat menentukan perilaku tingkat produk dengan menetapkan instruksi sistem, terpisah dari perintah yang diberikan oleh pengguna akhir.

Anda dapat menggunakan instruksi sistem dengan berbagai cara, termasuk:

- Menentukan persona atau peran (untuk chatbot, misalnya)
- Menentukan format keluaran (Markdown, YAML, dll.)
- Menentukan gaya dan nada keluaran (misalnya, verbositas, formalitas, dan tingkat membaca target)
- Menentukan tujuan atau aturan untuk tugas (misalnya, mengembalikan cuplikan kode tanpa penjelasan lebih lanjut)
- Memberikan konteks tambahan untuk perintah (misalnya, batas pengetahuan)



> **Ingat**: Kita menetapkan instruksi saat menginisialisasi model, lalu instruksi tersebut tetap ada selama semua interaksi dengan model.

In [ ]:
#hitung jumlah token dari seluruh respon
display(model.count_tokens(response_result))

total_tokens: 129

In [ ]:
#Hitung jumlah token perkata
display(model.count_tokens('AI'))
display(model.count_tokens('Cerdas'))
display(model.count_tokens('Kecerdasaran'))

total_tokens: 30

total_tokens: 31

total_tokens: 33

ingat kita masih menggunakan `system_instruction` yang mengakibatkan jumlah token pada `system_instruction` akan ditambahkan dengan prompt dan hasil respon.

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",)
  #tanpa instruksi sistem

response = model.generate_content(
    "Apa itu AI?",
    generation_config=genai.types.GenerationConfig(
        max_output_tokens=20,
        temperature=1.0,
        top_k=5,
        top_p=1.0,
    ),
)

response_result= response.text
print(response_result)

AI, atau **Kecerdasan Buatan**, adalah simulasi proses kecerdasan manusia


In [ ]:
#hitung jumlah token dari seluruh respon
display(model.count_tokens(response_result))

total_tokens: 20

In [ ]:
#Hitung jumlah token perkata
display(model.count_tokens('AI'))
display(model.count_tokens('Cerdas'))
display(model.count_tokens('Kecerdasaran'))

total_tokens: 1

total_tokens: 2

total_tokens: 4

Sekarang kita akan coba menerapkan **CoT**

In [ ]:
instruction= '''
Q: Roger memiliki 5 bola tenis.
Dia membeli 2 kaleng bola tenis lagi.
Setiap kaleng berisi 3 bola tenis.
Berapa banyak bola tenis yang dia miliki sekarang?

A: Roger awalnya memiliki 5 bola,
emudian membeli 2 kaleng berisi masing-masing 3 bola tenis,
sehingga 2x3 = 6 bola tenis. 5 + 6 = 11.
Jadi, jawabannya adalah 11.
'''

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=instruction)

In [ ]:
#kirim permintaan
input='''Jaka memiliki 23 apel.
Jika mereka menggunakan 20 untuk membuat makan siang dan membeli 6 lagi,
berapa banyak apel yang Jaka miliki sekarang?
'''
response = model.generate_content(input)
Markdown(response.text)

Berikut langkah-langkahnya:

1. **Mulai dengan jumlah awal:** Jaka memiliki 23 apel.
2. **Kurangi apel yang digunakan:** Jaka menggunakan 20 apel, jadi 23 - 20 = 3 apel tersisa.
3. **Tambahkan apel yang dibeli:** Jaka membeli 6 apel lagi, jadi 3 + 6 = 9 apel.

**Jawaban:** Jaka sekarang memiliki 9 apel.


In [ ]:
from IPython.display import Markdown
#Create Tree of Tought Prompting
prompt_tot = """
Imagine three different experts are answering this question.

All experts will write down 1 step of their thinking, then share it with the group.

Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.

after all expert reach step 3, conclude the answers from all experts.

The question:
"""

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=prompt_tot
  )

In [ ]:
#kirim permintaan
question = "why earth rotate?"
response = model.generate_content(question)
Markdown(response.text)

**Expert 1 (Planetary Scientist):**

**Step 1:** The Earth's rotation is a leftover from its formation.  The cloud of gas and dust that collapsed to form the Sun and planets had an inherent angular momentum – a measure of its rotation.  As this cloud collapsed, the conservation of angular momentum caused it to spin faster, much like a figure skater pulling in their arms.

**Step 2:**  This initial rotation was amplified during the accretion process, where dust and gas particles collided and stuck together, forming planetesimals and eventually the Earth.  Collisions between these bodies further influenced the Earth's rotation rate and axis.

**Step 3:** While tidal forces from the Moon and Sun slowly decelerate the Earth's rotation, the initial angular momentum from the solar nebula remains the fundamental reason for Earth's rotation.


**Expert 2 (Physicist):**

**Step 1:**  The Earth rotates due to the conservation of angular momentum.  This principle states that the total angular momentum of a system remains constant unless acted upon by an external torque.

**Step 2:** During the formation of the solar system, the initial cloud of gas and dust had some net angular momentum.  As this cloud collapsed under gravity, its rotation increased to conserve angular momentum.

**Step 3:** This initial angular momentum was transferred to the accreting Earth, resulting in its continued rotation.  While external forces like tidal interactions with the Moon cause slight changes to the rotation rate, the underlying cause remains the initial angular momentum.


**Expert 3 (Astronomer):**

**Step 1:** The Earth's rotation is a consequence of the angular momentum it inherited from the solar nebula, the giant rotating cloud of gas and dust from which our solar system formed.

**Step 2:**  This angular momentum wasn't evenly distributed; some regions rotated faster than others. The clumping together of matter to form the Earth resulted in a concentration of this angular momentum.

**Step 3:**  Gravitational interactions with other planets and the Sun have slightly altered Earth's rotational speed and axis over billions of years, but the initial spin from the nebula remains the primary reason for the Earth's rotation.


**Conclusion:**

All three experts agree that the Earth's rotation is primarily a consequence of the conservation of angular momentum from the initial solar nebula.  The rotation started during the formation of the solar system and was amplified by the accretion process and collisions. While external forces cause slight modifications, the fundamental reason remains the initial angular momentum.


In [ ]:
from IPython.display import Markdown
#Create Tree of Tought Prompting
prompt_tot = """
"""

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=prompt_tot
  )

In [ ]:
#kirim permintaan
question = "why earth rotate?"
response = model.generate_content(question)
Markdown(response.text)

The Earth rotates because it formed within a large, rotating cloud of gas and dust called the solar nebula.  As this nebula collapsed under its own gravity, it began to spin faster, much like a figure skater pulling their arms in.  This initial rotation was conserved as the Earth and other planets formed.  There's no external force strong enough to significantly slow it down, and the Earth's rotation continues due to the principle of conservation of angular momentum.


# Safety Settings (Setelan keamanan)


Argumen `safety_settings` memungkinkan Anda mengonfigurasi apa yang diblokir dan diizinkan oleh model baik dalam prompt maupun respons. Secara default, setelan keamanan memblokir konten dengan probabilitas **MEDIUM** dan/atau **HIGH** sebagai konten yang tidak aman di semua dimensi. Pelajari lebih lanjut tentang [Setelan keamanan](https://ai.google.dev/docs/safety_setting).

API Gemini mengkategorikan tingkat kemungkinan konten yang tidak aman sebagai HIGH, MEDIUM, LOW, atau NEGLIGIBLE.

**API Gemini memblokir konten berdasarkan kemungkinan konten tersebut tidak aman dan bukan tingkat keparahannya**. Hal ini penting untuk dipertimbangkan karena beberapa konten mungkin memiliki kemungkinan kecil untuk tidak aman meskipun tingkat keparahan bahayanya mungkin masih tinggi. Misalnya, bandingkan kalimat berikut:

- Robot itu meninjuku.
- Robot itu menebasku.

Kalimat pertama mungkin menghasilkan kemungkinan yang lebih tinggi untuk menjadi tidak aman, tetapi Anda mungkin menganggap kalimat kedua memiliki tingkat keparahan yang lebih tinggi dalam hal kekerasan. Mengingat hal ini, penting bagi Anda untuk menguji dan mempertimbangkan dengan saksama tingkat pemblokiran yang tepat yang diperlukan untuk mendukung kasus penggunaan utama Anda sekaligus meminimalkan kerugian bagi pengguna akhir.

Jenis kategori untuk `safety_settings` yang tersedia dapat dilihat [disini](https://ai.google.dev/api/generate-content#v1beta.HarmCategory)

In [ ]:
# kategori yang berbahaya
from google.generativeai.types import HarmCategory, HarmBlockThreshold

model = genai.GenerativeModel(model_name='gemini-1.5-flash-002')
response = model.generate_content(
    'buatlah konten berita dia adalah pencuri dan pembunuh',
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }
)
response.text

'Saya tidak bisa membuat konten berita yang menggambarkan seseorang sebagai pencuri dan pembunuh tanpa bukti yang kuat dan sah.  Menuduh seseorang melakukan kejahatan serius tanpa bukti adalah fitnah dan dapat memiliki konsekuensi hukum yang serius.\n\nJika Anda memiliki informasi tentang kejahatan yang telah dilakukan, Anda harus menghubungi penegak hukum. Mereka adalah orang yang tepat untuk menyelidiki situasi dan mengambil tindakan yang sesuai.  Hanya mereka yang memiliki wewenang untuk menyelidiki dan memutuskan apakah seseorang bersalah atau tidak.\n\nSaya dirancang untuk menjadi asisten yang bertanggung jawab dan etis, dan membuat konten yang menuduh seseorang melakukan kejahatan tanpa bukti melanggar prinsip-prinsip tersebut.\n'

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Saya tidak bisa membuat konten berita yang menggambarkan seseorang sebagai pencuri dan pembunuh tanpa bukti yang kuat dan sah.  Menuduh seseorang melakukan kejahatan serius tanpa bukti adalah fitnah dan dapat memiliki konsekuensi hukum yang serius.\n\nJika Anda memiliki informasi tentang kejahatan yang telah dilakukan, Anda harus menghubungi penegak hukum. Mereka adalah orang yang tepat untuk menyelidiki situasi dan mengambil tindakan yang sesuai.  Hanya mereka yang memiliki wewenang untuk menyelidiki dan memutuskan apakah seseorang bersalah atau tidak.\n\nSaya dirancang untuk menjadi asisten yang bertanggung jawab dan etis, dan membuat konten yang menuduh seseorang melakukan kejahatan tanpa bukti melanggar prinsip-prinsip tersebut.\n"
              }
      

# Function Calling

Dalam konteks Model Bahasa Besar (LLM), **function calling** merujuk pada kemampuan model untuk memanggil fungsi eksternal atau mengakses alat tambahan selama proses penalaran atau untuk menghasilkan hasil yang lebih baik. Dengan kata lain, LLM bisa berinteraksi dengan kode atau sistem lain untuk melakukan tugas yang lebih spesifik, seperti melakukan kalkulasi, mengakses database, atau menjalankan skrip di luar model itu sendiri.



Referensi materi untuk Function Calling:
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb
- https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.ipynb
- https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python
- https://codelabs.developers.google.com/codelabs/gemini-function-calling#0

In [ ]:
from google.genai import types
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


operation_tools = [add, subtract, multiply, divide]

In [ ]:
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)
chat = client.chats.create(
    model = "gemini-1.5-flash",
    config = {
        "tools": operation_tools,
        "automatic_function_calling": {"disable": False} # This line is not needed as automatic_function_calling is enabled by default
    }
)

In [ ]:
response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

'57 cats * 44 mittens/cat = 2508 mittens.  There are a total of 2508 mittens.\n'

In [ ]:
from IPython.display import Markdown, display

for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)

###user:

I have 57 cats, each owns 44 mittens, how many mittens is that in total?

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'b': 44, 'a': 57} name='multiply' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='multiply' response={'result': 2508} }
--------------------------------------------------------------------------------


###model:

57 cats * 44 mittens/cat = 2508 mittens.  There are a total of 2508 mittens.


--------------------------------------------------------------------------------


In [ ]:
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [ ]:
from google.genai import types

config = types.GenerateContentConfig(tools=[set_light_values])

In [ ]:
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents='Turn the lights down to a movie watching level'
)
print(response.text)

What brightness level would you consider appropriate for watching a movie? 



In [ ]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='What brightness level would you like to set for watching a movie?')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.34281209798959583, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-2.0-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=13, prompt_token_count=107, total_token_count=120), automatic_function_calling_history=[], parsed=None)

In [ ]:
# Use the chat interface
chat = client.chats.create(model='gemini-2.0-flash', config=config)
response = chat.send_message('Turn the lights down to a movie watching level')
print(response.text)

What brightness level would you like to set for watching a movie?



# Latihan

Sekarang cobalah lakukan latihan untuk menerapkan beberapa teknik prompting serta dengan konfigurasi temperature, top_k dan top_p yang berbeda-beda.

In [ ]:
import google.generativeai as genai

prompt_sentiment_analysis = """
Anda adalah AI yang menganalisis sentimen teks
Klasifikasikan teks ke dalam salah satu kategori berikut:
- Positif
- Negatif
- Netral

Berikan juga penjelasan singkat mengapa teks diklasifikasikan ke dalam kategori tersebut.
"""

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=prompt_sentiment_analysis
  )

question = "besok ada acara di desa"
response = model.generate_content(
    question,
    generation_config=genai.types.GenerationConfig(
        max_output_tokens=200,
        temperature=1.0,
        top_k=5,
        top_p=1.0,
    ),
  )
Markdown(response.text)

Netral.

Teks tersebut hanya menyatakan fakta adanya acara di desa besok, tanpa mengungkapkan sentimen positif maupun negatif.  Tidak ada kata-kata atau frasa yang menunjukkan emosi atau penilaian terhadap acara tersebut.
